<a href="https://colab.research.google.com/github/haodongzhang0118/Stock-Trading-Ensemble-Model/blob/main/DataRetrieve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ta
!pip install wrds
!pip install swig
!pip install -q condacolab
import condacolab
condacolab.install()
!apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib/
!./configure --prefix=/usr
!make
!make install
!pip install TA-Lib

In [117]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import talib
from finrl.config_tickers import DOW_30_TICKER
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv

In [118]:
print(DOW_30_TICKER)
start_date = "2021-01-01"
end_date = "2023-09-01"

['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [119]:
# Initialize a list to store the data
all_data = []

for ticker in DOW_30_TICKER:
    # Download data
    if ticker == "DOW":
      continue
    data = yf.download(ticker, start=start_date, end=end_date)

    # Calculate Technical Indicators
    data['MACD'], data['MACDSignal'], data['MACDHist'] = talib.MACD(data['Close'], fastperiod=2, slowperiod=3, signalperiod=4)
    data['RSI'] = talib.RSI(data['Close'], timeperiod=14)
    data['CCI'] = talib.CCI(data['High'], data['Low'], data['Close'], timeperiod=14)
    data['ADX'] = talib.ADX(data['High'], data['Low'], data['Close'], timeperiod=14)
    data.rename(columns={'Close': 'close'}, inplace=True)

    # Add ticker and reset index
    data['tic'] = ticker
    data.reset_index(inplace=True)

    # Append to the list
    all_data.append(data)

# Combine into a single DataFrame
combined_data = pd.concat(all_data)

# Sort by date and reset index
combined_data.sort_values(by='Date', inplace=True)
combined_data.rename(columns={'Date': 'date'}, inplace=True)
combined_data.reset_index(drop=True, inplace=True)

# Select relevant columns
final_df = combined_data[['date', 'tic', 'close', 'Adj Close', 'MACD', 'RSI', 'CCI', 'ADX']]

final_df.reset_index()
final_df['index'] = final_df.groupby('date').ngroup()
final_df.set_index('index', inplace=True)

# Show the final DataFrame
final_df.tail()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,date,tic,close,Adj Close,MACD,RSI,CCI,ADX
index,,,,,,,,
669,2023-08-31,CVX,161.100006,159.428802,0.226501,54.762086,49.722507,7.917108
669,2023-08-31,GS,327.709991,327.709991,0.149036,44.767633,23.791005,22.871932
669,2023-08-31,NKE,101.709999,101.369995,0.195521,40.167683,-12.716591,18.229722
669,2023-08-31,CAT,281.130005,279.684814,0.734209,60.518695,112.800107,23.234817
669,2023-08-31,DIS,83.680000,83.680000,-0.079953,39.966518,-73.639173,12.936917


In [120]:
reward_scaling = 1e-4
Indicators = ['Adj Close', 'MACD', 'RSI', 'CCI', 'ADX']
stock_dim = len(final_df.tic.unique())
state_space = 1 + 2 * stock_dim + len(Indicators) * stock_dim
InitialAmount = 1000000
num_stock_holders = [0] * stock_dim
buy_cost = [0.001] * stock_dim
sell_cost = [0.001] * stock_dim
print("""There are {} stocks in the dataset.
The state space we used for training is {}.
The indicators we used is {}.
The reward scaling is {}.
The initial amount is {}.
The share for each storck in the beginning is 0.
Cost of buying and selling is 0.001.
""".format(stock_dim, state_space, Indicators, reward_scaling, InitialAmount))

There are 29 stocks in the dataset. 
The state space we used for training is 204. 
The indicators we used is ['Adj Close', 'MACD', 'RSI', 'CCI', 'ADX'].
The reward scaling is 0.0001.
The initial amount is 1000000.
The share for each storck in the beginning is 0.
Cost of buying and selling is 0.001.



In [121]:
env_arg = {"hmax": 100,
           "initial_amount": InitialAmount,
           "num_stock_shares": num_stock_holders,
           "buy_cost_pct": buy_cost,
           "sell_cost_pct": sell_cost,
           "state_space": state_space,
           "stock_dim": stock_dim,
           "tech_indicator_list": Indicators,
           "action_space": stock_dim,
           "reward_scaling": reward_scaling}
env = StockTradingEnv(df=final_df, **env_arg)
env, _ = env.get_sb_env()
type(env)

stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv